## create_datasets를 먼저 돌리고 실행

In [1]:
import pandas as pd

## Size 요소 가져오기

In [2]:
stock_data = pd.read_hdf('assets.h5', 'finance_datareader/prices')
market_cap_rank_data = stock_data[['marcap', 'rank']]

In [3]:
total_asset = pd.read_csv('IFRS/TotalAssets.csv', encoding='CP949')

In [4]:
# 종목코드를 6자리로 맞춰줌 -> 파이썬 고질병 '000660'을 숫자로 자동 인식하면 앞의 000을 날려버림 이거를 string으로 변경해서 복구
ticker = total_asset['거래소코드'].apply("{0:0>6}".format)

In [5]:
# 금융업 제외한 재무제표랑 market cap 데이터랑 겹치는 종목 추출
intersect_ticker = market_cap_rank_data.unstack('date').index.intersection(ticker)

In [6]:
size_factor = (market_cap_rank_data
                .unstack('date')
                .loc[intersect_ticker,:].stack('date')
                .reorder_levels(['date','ticker'])
                .sort_index())

In [7]:
size_factor = size_factor.astype('int64')

In [8]:
# 일자별 시가총액 순위 데이터
size_factor

marcap  rank
date       ticker                     
1995-05-02 000020    76796000000   191
           000040    24826762500   363
           000050    84854000000   179
           000070   247896000000    58
           000080   166793251500    81
...                          ...   ...
2021-05-21 363280  1070573539750   200
           375500  2623108999000   116
           378850   119469888670   640
           900140   354060857320   384
           950210  1730769264000   149

[3550007 rows x 2 columns]

## B/M Book to Market Ratio 구하기
- 시가총액은 연도 말을 기준으로 한다.
- 재무제표는 사업보고서이고 공시 날짜는 6월 1일로 통일한다.

In [9]:
def get_ifrs_data(data_path: str):
    data = pd.read_csv(data_path, encoding='CP949')
    data.fillna(0, inplace=True)
    data['거래소코드'] = data['거래소코드'].apply("{0:0>6}".format)
    data['회계년도'] = pd.to_datetime(data['회계년도'])
    data = data.set_index(['회계년도','거래소코드']).sort_index()
    # 회계년도 안맞는 것들 resample을 이용해 연말로 회계년도 통일
    data = data.unstack('거래소코드').resample('Y').last().stack('거래소코드')
    data.index = data.index.set_levels(data.index.levels[0].to_period('M'), level=0)
    col_list = data.columns
    before_ifrs = data.loc[idx[:'2006',:],:][col_list[-2]]
    after_ifrs = data.loc[idx['2007':,:],:][col_list[-1]]
    data = pd.concat([before_ifrs, after_ifrs]).to_frame()
    data.columns = [col_list[-2]]
    data.index.names = ['date','ticker']
    data.sort_index(inplace=True)
    return data.copy()

In [10]:
idx = pd.IndexSlice

In [11]:
common_stock_capital = get_ifrs_data('IFRS/CommonStock.csv')

In [13]:
common_stock_capital

보통주자본금(천원)
date    ticker             
1981-12 000020    2808000.0
        000040    3450000.0
        000050    2400000.0
        000070    9500000.0
        000080    3000000.0
...                     ...
2020-12 363280   18750451.0
        375500          0.0
        378850          0.0
        900140          0.0
        950210  152779419.0

[20876 rows x 1 columns]

In [14]:
capital_surplus = get_ifrs_data('IFRS/CapitalSurplus.csv')

In [16]:
capital_surplus

자본잉여금(*)(천원)
date    ticker              
1981-12 000020        4410.0
        000040     1903860.0
        000050    12797820.0
        000070     9538010.0
        000080     8837667.0
...                      ...
2020-12 363280   528596133.0
        375500           0.0
        378850           0.0
        900140           0.0
        950210           0.0

[20876 rows x 1 columns]

In [17]:
retained_earnings = get_ifrs_data('IFRS/RetainedEarnings.csv')

In [18]:
retained_earnings

이익잉여금(*)(천원)
date    ticker              
1981-12 000020     2567147.0
        000040      343796.0
        000050           0.0
        000070     6452616.0
        000080     1002994.0
...                      ...
2020-12 363280    -2390485.0
        375500           0.0
        378850           0.0
        900140           0.0
        950210   -21560969.0

[20876 rows x 1 columns]

In [19]:
deferred_tax_liabilities = get_ifrs_data('IFRS/DeferredTaxLiabilities.csv')

In [20]:
market_cap_data = stock_data['marcap'].to_frame()

In [21]:
market_cap_data = market_cap_data.unstack('ticker').resample('Y').last().stack('ticker')

In [22]:
market_cap_data.index = market_cap_data.index.set_levels(market_cap_data.index.levels[0].to_period('M'), level=0)

In [23]:
market_cap_data

marcap
date    ticker              
1995-12 000020  6.390900e+10
        000030  1.317500e+12
        000040  2.919949e+10
        000050  6.760600e+10
        000060  2.385240e+11
...                      ...
2021-12 378850  1.194699e+11
        380440  1.077090e+11
        383220  3.160334e+12
        900140  3.540609e+11
        950210  1.730769e+12

[18516 rows x 1 columns]

In [24]:
book_to_market_data = pd.concat([common_stock_capital, capital_surplus, retained_earnings, deferred_tax_liabilities, market_cap_data], axis=1)
book_to_market_data.sort_index(inplace=True)

In [25]:
book_to_market_data

보통주자본금(천원)  자본잉여금(*)(천원)  이익잉여금(*)(천원)  이연법인세부채(천원)  \
date    ticker                                                        
1981-12 000020   2808000.0        4410.0     2567147.0          0.0   
        000040   3450000.0     1903860.0      343796.0          0.0   
        000050   2400000.0    12797820.0           0.0          0.0   
        000070   9500000.0     9538010.0     6452616.0          0.0   
        000080   3000000.0     8837667.0     1002994.0          0.0   
...                    ...           ...           ...          ...   
2021-12 378850         NaN           NaN           NaN          NaN   
        380440         NaN           NaN           NaN          NaN   
        383220         NaN           NaN           NaN          NaN   
        900140         NaN           NaN           NaN          NaN   
        950210         NaN           NaN           NaN          NaN   

                      marcap  
date    ticker                
1981-12 000020           NaN  
        000040           NaN  
        000050           NaN  
        000070           NaN  
        000080           NaN  
...                      ...  
2021-12 378850  1.194699e+11  
        380440  1.077090e+11  
        383220  3.160334e+12  
        900140  3.540609e+11  
        950210  1.730769e+12  

[25086 rows x 5 columns]

In [26]:
# 시가총액이 있는 년도부터 시작
book_to_market_data = book_to_market_data.loc[idx['1995':'2021',:],:]

In [27]:
# 결측치 제거
book_to_market_data.dropna(inplace=True)

<ipython-input-27-f2b34a7bd11a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_to_market_data.dropna(inplace=True)


### Book to Market Ratio 구하기

In [39]:
# B/M 계산
columns_list = book_to_market_data.columns
book_value = book_to_market_data.loc[:,columns_list[:-1]].sum(axis=1)
book_to_market_ratio = book_value.div(book_to_market_data['marcap']).to_frame()
book_to_market_ratio.columns = ['BM']


### 회계년도랑 공시년도가 차이가 나므로 회계년도에서 6개월 뒤로 미룬다.
- 년말 12월 기준의 회계가 있어도 공시하기까지 시간이 걸린다. 이부분을 고려

In [103]:
def offset_6_month(data: pd.DataFrame) -> pd.DataFrame:
    '''
        Input DataFrame E.g
        Multi Index DataFrame
                      price
        date  ticker
        1995  000660  1000
              005930  20000
              003229  3004000
    '''
    date_index = data.index.levels[0].to_timestamp() + pd.DateOffset(months=6)
    data.index = data.index.set_levels(date_index.to_period('M'), level=0)
    return data 

In [40]:
date_index = book_to_market_ratio.index.levels[0].to_timestamp() + pd.DateOffset(months=6)

In [41]:
book_to_market_ratio.index = book_to_market_ratio.index.set_levels(date_index.to_period('M'), level=0)

In [42]:
book_to_market_ratio

BM
date    ticker          
1996-06 000020  0.000646
        000040  0.001656
        000050  0.000926
        000070  0.001059
        000080  0.001073
...                  ...
2021-06 344820  0.002336
        352820  0.000212
        353200  0.001031
        363280  0.000537
        900140  0.000000

[14306 rows x 1 columns]

## 수익성 지표 계산
- OP = 영업이익 / 자기자본 장부가치
    + 영업이익 = 매출액 - 매출원가 - 이자비용 - 판관비

#### 매출액

In [44]:
net_sales = get_ifrs_data('IFRS/NetSales.csv')

In [45]:
net_sales

매출액(영업수익)(*)(천원)
date    ticker                  
1981-12 000020        26964281.0
        000040        48075570.0
        000050        52868914.0
        000070       223628100.0
        000080        93182984.0
...                          ...
2020-12 363280               0.0
        375500               0.0
        378850               0.0
        900140               0.0
        950210               0.0

[20876 rows x 1 columns]

#### 매출원가

In [46]:
cost_of_sales = get_ifrs_data('IFRS/CostOfSales.csv')

In [47]:
cost_of_sales

매출원가(*)(천원)
date    ticker             
1981-12 000020   16814161.0
        000040   42802676.0
        000050   46724109.0
        000070  189372232.0
        000080   75451679.0
...                     ...
2020-12 363280          0.0
        375500          0.0
        378850          0.0
        900140          0.0
        950210          0.0

[20876 rows x 1 columns]

#### 이자비용

In [48]:
interest_expense = get_ifrs_data('IFRS/InterestExpenses.csv')

In [49]:
interest_expense

이자비용(천원)
date    ticker          
1981-12 000020       0.0
        000040       0.0
        000050       0.0
        000070       0.0
        000080       0.0
...                  ...
2020-12 363280       0.0
        375500       0.0
        378850       0.0
        900140       0.0
        950210       0.0

[20876 rows x 1 columns]

### 판매비와 관리비

In [50]:
selling_and_admin_expense = get_ifrs_data('IFRS/Selling_and_administrative_expenses.csv')

In [51]:
selling_and_admin_expense

판매비와관리비(영업비용)(*)(천원)
date    ticker                      
1981-12 000020             6385484.0
        000040             2264367.0
        000050             2939748.0
        000070            10461231.0
        000080            11557635.0
...                              ...
2020-12 363280             2265873.0
        375500                   0.0
        378850                   0.0
        900140                   0.0
        950210            10708804.0

[20876 rows x 1 columns]

In [69]:
op_data = pd.concat([net_sales, cost_of_sales, interest_expense, selling_and_admin_expense], axis=1)

In [71]:
op_data.head(5)

매출액(영업수익)(*)(천원)  매출원가(*)(천원)  이자비용(천원)  판매비와관리비(영업비용)(*)(천원)
date    ticker                                                               
1981-12 000020        26964281.0   16814161.0       0.0             6385484.0
        000040        48075570.0   42802676.0       0.0             2264367.0
        000050        52868914.0   46724109.0       0.0             2939748.0
        000070       223628100.0  189372232.0       0.0            10461231.0
        000080        93182984.0   75451679.0       0.0            11557635.0

In [72]:
col_list = op_data.columns

In [85]:
# 위의 식대로 계산
op = (op_data[col_list[0]] - op_data[col_list[1:]].sum(axis=1)).div(book_value).to_frame()

In [86]:
op.columns = ['OP']
op.dropna(inplace=True)

In [104]:
op = offset_6_month(op)

In [105]:
op 

OP
date    ticker          
1996-06 000020  0.615225
        000040 -0.132015
        000050  0.121340
        000070  0.229497
        000080  0.263004
...                  ...
2021-06 339770  0.198420
        344820  0.026108
        352820  0.071987
        353200  0.018592
        363280 -0.004065

[14260 rows x 1 columns]

## 자본투자
- t-1년 12월 말의 총자산에서 t-2년 12월 말의 총 자산을 차감한 총자산증가액
    + inv = TotalAsset(t-1) / TotalAsset(t-2) - 1

In [88]:
total_asset = get_ifrs_data('IFRS/TotalAssets.csv')

In [91]:
total_assett_unstack = total_asset.unstack('ticker')

In [97]:
inv = (total_assett_unstack / total_assett_unstack.shift(1)).stack('ticker') - 1

In [98]:
inv

자산(*)(천원)
date    ticker           
1982-12 000040   0.150404
        000050   0.033943
        000100   0.524996
        000120   0.108065
        000140   0.154147
...                   ...
2020-12 336260   0.593889
        336370   0.508118
        339770   0.519012
        352820   5.267460
        361610   1.137762

[19811 rows x 1 columns]